In [1]:
# Import the required classes
from pyspark import SparkConf
import pandas as pd
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.rdd import RDD
from pyspark.sql.types import IntegerType
import numpy as np

# Part A : Working with RDDs and DataFrames (80%)
## 1. Working with RDD (25%)  
### 1.1 Data Preparation and Loading (5%)

In [2]:
# 1.
# Set working processors to number of logical ocres on the machine
master = "local[*]"
# Name to be shown on Spark cluster UI page
app_name = "Assignment 1: Analyzing Road Crash Data"

# setting configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Import SparkContext and SparkSession classes 
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Building SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [3]:
# 2.
# Read all files ending in Units.csv
rdd_units = sc.textFile("*Units.csv")

In [4]:
# 3.
# Read all files ending in Crash.csv
rdd_crashes = sc.textFile("*Crash.csv")

In [5]:
#4.
# Storing first row 
header = rdd_units.first()
# Removing the first row
rdd_units = rdd_units.filter(lambda x: x != header)
# Number of rows
rdd_units.count()

153854

In [6]:
# 4.
# Displaying the first 10 records
rdd_units.take(10)

['"2016-1-15/08/2019","01",0,"SA","OMNIBUS","2011","North","Male","056","SA","HR","Full","Not Towing","Straight Ahead","010","5121",,',
 '"2016-1-15/08/2019","02",1,,"Pedestrian on Road",,"East","Male","072",,,,,"Walking on Road",,"5084",,',
 '"2016-2-15/08/2019","01",0,"SA","Motor Cars - Sedan","2004","Unknown","Female","023","SA","C ","Full","Not Towing","Straight Ahead","001","5087",,',
 '"2016-2-15/08/2019","02",0,"SA","Station Wagon","2008","Unknown","Male","040","SA","C ","Full","Not Towing","Straight Ahead","001","5084",,',
 '"2016-3-15/08/2019","01",0,"SA","RIGID TRUCK LGE GE 4.5T","1990","South","Unknown","XXX","SA","MR","Provisional 2","Not Towing","Straight Ahead","001","5115",,',
 '"2016-3-15/08/2019","02",0,"SA","Panel Van","2013","South","Male","023","SA","C ","Full","Not Towing","Straight Ahead","001","5110",,',
 '"2016-4-15/08/2019","01",0,"SA","Station Wagon","2002","East","Female","033","SA","C ","Full","Not Towing","Straight Ahead","001","5169",,',
 '"2016-4-15/08/20

In [7]:
#4.
# Storing first row 
header = rdd_crashes.first()
# Removing the first row
rdd_crashes = rdd_crashes.filter(lambda x: x != header)
# Number of rows
rdd_crashes.count()

72006

In [8]:
# 4.
# Displaying the first 10 records
rdd_crashes.take(10)

['"2019-1-8/07/2020","2 Metropolitan","HAMPSTEAD GARDENS","5086","CITY OF PORT ADELAIDE ENFIELD",2,0,0,0,0,2019,"June","Wednesday","11:15 am","060","Cross Road","Straight road","Level","Not Applicable","Sealed","Dry","Not Raining","Daylight","Right Angle","01","Driver Rider","1: PDO","Give Way Sign","","",1331810.03,1676603.26,"13318101676603"',
 '"2019-2-8/07/2020","2 Metropolitan","DRY CREEK","5094","CITY OF SALISBURY",2,0,0,0,0,2019,"January","Tuesday","12:49 am","090","Divided Road","Straight road","Level","Not Applicable","Sealed","Dry","Not Raining","Night","Rear End","02","Driver Rider","1: PDO","No Control","","",1328376.2,1682942.63,"13283761682943"',
 '"2019-3-8/07/2020","2 Metropolitan","MILE END","5031","CITY OF WEST TORRENS",2,1,0,0,1,2019,"January","Tuesday","12:00 am","060","Divided Road","Straight road","Level","Not Applicable","Sealed","Dry","Not Raining","Night","Hit Pedestrian","01","Driver Rider","2: MI","No Control","","",1325819.68,1670994.26,"13258201670994"',
 '

### 1.2 Data Partitioning in RDD (10%)

In [9]:
# 1.
# Default number of partitions for rdd_units
print("Number of partitions in rdd units:", rdd_units.getNumPartitions())
# Default number of partitions for rdd_crashes
print("Number of partitions in rdd crashes :", rdd_crashes.getNumPartitions())

Number of partitions in rdd units: 5
Number of partitions in rdd crashes : 5


In [10]:
# 2.a.
# Storing first row 
header = rdd_units.first()
# Removing the first row
rdd_units = rdd_units.filter(lambda x: x != header)
# Splitting different columns
rdd_units_1 = rdd_units.map(lambda x: x.split(","))
# Making Lic State as key and other columns as Value
rdd_units_2 = rdd_units_1.map(lambda x: (x.pop(9), x[:]))

In [11]:
# 2.b.
# Function to implement hash partitioning
def hash_function(key):
    if key == '"SA"':
        partition = 0
    else:
        partition = 1
    return partition

In [12]:
# 2.b.
# hash partitioning
rdd_hash = rdd_units_2.partitionBy(2, hash_function)

In [13]:
# 2.c.
# A Function to print number of records in each partition
def print_partitions(data):
    
    numPartitions = data.getNumPartitions()
    partitions = data.glom().collect()
    
    print(f"NUMBER OF PARTITIONS: {numPartitions}")
    for index, partition in enumerate(partitions):
        # show partition if it is not empty
        if len(partition) > 0:
            print(f"Partition {index}: {len(partition)} records")

In [14]:
# 2.c.
print_partitions(rdd_hash)

NUMBER OF PARTITIONS: 2
Partition 0: 109684 records
Partition 1: 44170 records


We observe that the number of records in partition 0 are nearly 2.5 times the number of records in partition 1. Therefore, the data is definitely skewed and we can say that there are a huge number of vehicles registered in SA state.

### 1.3 Query/Analysis (10%)

In [15]:
# 1.
# Extracting columns sex and age and removing quotes
agesex_rdd = rdd_units_1.map(lambda x: (x[7].strip('""'), x[8].strip('""')))
agesex_rdd.take(10)

[('Male', '056'),
 ('Male', '072'),
 ('Female', '023'),
 ('Male', '040'),
 ('Unknown', 'XXX'),
 ('Male', '023'),
 ('Female', '033'),
 ('Unknown', 'XXX'),
 ('Male', '042'),
 ('Male', '059')]

In [16]:
# 1.
# Filtering males and females
male = agesex_rdd.filter(lambda x: x[0]=='Male' and x[1]!='XXX')
female = agesex_rdd.filter(lambda x: x[0]=='Female' and x[1]!= 'XXX')

In [17]:
# 1.
# Type Casting string to integer
male_age = male.map(lambda x: int(x[1]))
# Average male age
male_age.mean()

40.97596029992008

In [18]:
# 1.
# Type Casting string to integer
female_age = female.map(lambda x: int(x[1]))
# Average female age
female_age.mean()

40.38729268862415

In [19]:
# 2.
# Extracting columns Veh Reg State, Unit Type, Veh Year and removing quotes
rdd_veh = rdd_units_1.map(lambda x: (x[3].strip('""'), x[4].strip('""'), x[5].strip('""')))

In [20]:
# 2.
# Removing veh year having null and xxxx values
rdd_veh = rdd_veh.filter(lambda x: x[2]!='XXXX' and x[2]!='')

In [21]:
# 2.
# Type Casting string to integer
rdd_veh = rdd_veh.map(lambda x: (x[0], x[1], int(x[2])))
rdd_veh.take(5)

[('SA', 'OMNIBUS', 2011),
 ('SA', 'Motor Cars - Sedan', 2004),
 ('SA', 'Station Wagon', 2008),
 ('SA', 'RIGID TRUCK LGE GE 4.5T', 1990),
 ('SA', 'Panel Van', 2013)]

In [22]:
# 2.
# Get max by value in index 2 (veh year)
veh_max = rdd_veh.max(key=lambda x: x[2]) 
# Newest vehicle
print(veh_max)

('SA', 'Station Wagon', 2019)


In [23]:
# 2.
# Get min by value in index 2 (veh year)
veh_min = rdd_veh.min(key=lambda x: x[2]) 
# Oldest vehicle
print(veh_min)

('VIC', 'Motor Cycle', 1900)


## 2. Working with DataFrames (35%)
###  2.1 Data Preparation and Loading (5%)

In [24]:
# 1.
# Loading all files ending with Units.csv into a single df
df_units = spark.read.csv("*Units.csv", header="true")
# Loading all files ending with Crash.csv into a single df
df_crashes = spark.read.csv("*Crash.csv", header="true")

In [25]:
# 2.
# Displaying schema for units df
df_units.printSchema()

root
 |-- REPORT_ID: string (nullable = true)
 |-- Unit No: string (nullable = true)
 |-- No Of Cas: string (nullable = true)
 |-- Veh Reg State: string (nullable = true)
 |-- Unit Type: string (nullable = true)
 |-- Veh Year: string (nullable = true)
 |-- Direction Of Travel: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Lic State: string (nullable = true)
 |-- Licence Class: string (nullable = true)
 |-- Licence Type: string (nullable = true)
 |-- Towing: string (nullable = true)
 |-- Unit Movement: string (nullable = true)
 |-- Number Occupants: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Rollover: string (nullable = true)
 |-- Fire: string (nullable = true)



In [26]:
# 2.
# Displaying schema for crashes df
df_crashes.printSchema()

root
 |-- REPORT_ID: string (nullable = true)
 |-- Stats Area: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- LGA Name: string (nullable = true)
 |-- Total Units: string (nullable = true)
 |-- Total Cas: string (nullable = true)
 |-- Total Fats: string (nullable = true)
 |-- Total SI: string (nullable = true)
 |-- Total MI: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Area Speed: string (nullable = true)
 |-- Position Type: string (nullable = true)
 |-- Horizontal Align: string (nullable = true)
 |-- Vertical Align: string (nullable = true)
 |-- Other Feat: string (nullable = true)
 |-- Road Surface: string (nullable = true)
 |-- Moisture Cond: string (nullable = true)
 |-- Weather Cond: string (nullable = true)
 |-- DayNight: string (nullable = true)
 |-- Crash Type: string (nullable = true)
 |-- U

### 2.2 Query/Analysis (15%)

In [27]:
# 1.
# using cast function to change the datatype of Total Cas column
df_crashes = df_crashes.withColumn('Total Cas', F.col('Total Cas').cast(IntegerType()))
# display the datatype of all columns
display(df_crashes)

DataFrame[REPORT_ID: string, Stats Area: string, Suburb: string, Postcode: string, LGA Name: string, Total Units: string, Total Cas: int, Total Fats: string, Total SI: string, Total MI: string, Year: string, Month: string, Day: string, Time: string, Area Speed: string, Position Type: string, Horizontal Align: string, Vertical Align: string, Other Feat: string, Road Surface: string, Moisture Cond: string, Weather Cond: string, DayNight: string, Crash Type: string, Unit Resp: string, Entity Code: string, CSEF Severity: string, Traffic Ctrls: string, DUI Involved: string, Drugs Involved: string, ACCLOC_X: string, ACCLOC_Y: string, UNIQUE_LOC: string]

In [28]:
# 1.
# filtering adelaide suburbs where Total cas is greater than 3
df_adelaide = df_crashes.filter(df_crashes['Suburb'] == 'ADELAIDE').filter(df_crashes['Total Cas']>3)
print("Incidents:", df_adelaide.count())
df_adelaide.show()

Incidents: 4
+--------------------+----------+--------+--------+----------------+-----------+---------+----------+--------+--------+----+--------+--------+--------+----------+-------------+----------------+--------------+--------------------+------------+-------------+------------+--------+--------------+---------+------------+-------------+---------------+------------+--------------+----------+----------+--------------+
|           REPORT_ID|Stats Area|  Suburb|Postcode|        LGA Name|Total Units|Total Cas|Total Fats|Total SI|Total MI|Year|   Month|     Day|    Time|Area Speed|Position Type|Horizontal Align|Vertical Align|          Other Feat|Road Surface|Moisture Cond|Weather Cond|DayNight|    Crash Type|Unit Resp| Entity Code|CSEF Severity|  Traffic Ctrls|DUI Involved|Drugs Involved|  ACCLOC_X|  ACCLOC_Y|    UNIQUE_LOC|
+--------------------+----------+--------+--------+----------------+-----------+---------+----------+--------+--------+----+--------+--------+--------+----------+-

In [29]:
# 2.
# Displaying 10 events with highest casualities
df_crashes.sort("Total Cas", ascending=False).show(10)

+--------------------+--------------+---------------+--------+--------------------+-----------+---------+----------+--------+--------+----+--------+---------+--------+----------+-------------+--------------------+--------------+--------------+------------+-------------+------------+--------+-----------+---------+------------+-------------+---------------+------------+--------------+----------+----------+--------------+
|           REPORT_ID|    Stats Area|         Suburb|Postcode|            LGA Name|Total Units|Total Cas|Total Fats|Total SI|Total MI|Year|   Month|      Day|    Time|Area Speed|Position Type|    Horizontal Align|Vertical Align|    Other Feat|Road Surface|Moisture Cond|Weather Cond|DayNight| Crash Type|Unit Resp| Entity Code|CSEF Severity|  Traffic Ctrls|DUI Involved|Drugs Involved|  ACCLOC_X|  ACCLOC_Y|    UNIQUE_LOC|
+--------------------+--------------+---------------+--------+--------------------+-----------+---------+----------+--------+--------+----+--------+------

In [30]:
# 3.
# Total fatalities when grouped by Crash Type
df_crashes.groupBy("Crash Type")\
.agg(F.sum("Total Fats").alias("Total Fatalities"))\
.show()

+--------------------+----------------+
|          Crash Type|Total Fatalities|
+--------------------+----------------+
|           Roll Over|            57.0|
|  Hit Object on Road|             2.0|
|      Hit Pedestrian|            70.0|
|    Hit Fixed Object|           152.0|
|               Other|             2.0|
|          Side Swipe|            20.0|
|             Head On|            86.0|
|  Hit Parked Vehicle|             9.0|
|          Right Turn|            18.0|
|            Rear End|            16.0|
|          Hit Animal|             4.0|
|Left Road - Out o...|             1.0|
|         Right Angle|            45.0|
+--------------------+----------------+



In [31]:
# 4.
# Joining crash and units df on REPORT_ID
df_joint = df_units.join(df_crashes, df_crashes.REPORT_ID==df_units.REPORT_ID, how='outer')

In [32]:
# 4.
# Taking only unlicenced drivers 
df_unlicenced = df_joint.filter(df_joint['Licence Type']=='Unlicenced')

In [33]:
# 4.
# Total casualities for each suburb
df_unlicenced.groupBy("Suburb")\
.agg(F.sum("No Of Cas").alias("Total Casualities"))\
.sort("Total Casualities", ascending=False)\
.show()

+---------------+-----------------+
|         Suburb|Total Casualities|
+---------------+-----------------+
|      DRY CREEK|             11.0|
|       PROSPECT|             11.0|
|        ENFIELD|             11.0|
|      SALISBURY|             11.0|
|   ANDREWS FARM|             10.0|
| SALISBURY EAST|              8.0|
| ELIZABETH PARK|              8.0|
|SALISBURY DOWNS|              8.0|
|       ADELAIDE|              8.0|
|         BURTON|              8.0|
| NORTH ADELAIDE|              7.0|
|SALISBURY NORTH|              7.0|
|  MOUNT GAMBIER|              7.0|
|    COOBER PEDY|              7.0|
|   BEDFORD PARK|              6.0|
|ELIZABETH DOWNS|              6.0|
|   DAVOREN PARK|              6.0|
|  GILLES PLAINS|              6.0|
|          BURRA|              6.0|
|  MURRAY BRIDGE|              6.0|
+---------------+-----------------+
only showing top 20 rows



### 2.3 Severity Analysis (15%)

In [34]:
# 1.
# Total number of crashes for each severity level
df_crashes.groupBy("CSEF Severity").count().show()

+-------------+-----+
|CSEF Severity|count|
+-------------+-----+
|     4: Fatal|  451|
|        2: MI|21881|
|       1: PDO|46696|
|        3: SI| 2978|
+-------------+-----+



Severity level 1:PDO (property damage only)is the most common.

In [35]:
# 2.a.
# Total number of cases involving drugs
total = df_crashes.filter(df_crashes['Drugs Involved']=='Y').count()

# Count and percentage for cases involving drugs
df_crashes.filter(df_crashes['Drugs Involved']=='Y')\
.groupBy("CSEF Severity")\
.count()\
.withColumn('Percentage', (F.col('count') / total) * 100)\
.show()

+-------------+-----+------------------+
|CSEF Severity|count|        Percentage|
+-------------+-----+------------------+
|     4: Fatal|   82| 6.539074960127592|
|        2: MI|  749|59.728867623604465|
|       1: PDO|  176|14.035087719298245|
|        3: SI|  247|19.696969696969695|
+-------------+-----+------------------+



In [36]:
# 2.b.
# Total number of cases involving DUI/BAC
total = df_crashes.filter(df_crashes['DUI Involved']=='Y').count()

# Count and percentage for cases involving DUI/BAC
df_crashes.filter(df_crashes['DUI Involved']=='Y')\
.groupBy("CSEF Severity")\
.count()\
.withColumn('Percentage', (F.col('count') / total) * 100)\
.show()

+-------------+-----+------------------+
|CSEF Severity|count|        Percentage|
+-------------+-----+------------------+
|     4: Fatal|   79|  3.51423487544484|
|        2: MI|  737|  32.7846975088968|
|       1: PDO| 1173|52.179715302491104|
|        3: SI|  259| 11.52135231316726|
+-------------+-----+------------------+



In [37]:
# 2.c
# Total number of cases involving DUI/BAC and drugs
total = df_crashes.filter(df_crashes['Drugs Involved']=='Y')\
        .filter(df_crashes['DUI Involved']=='Y').count()

# Count and percentage for cases involving DUI/BAC and drugs
df_crashes.filter(df_crashes['Drugs Involved']=='Y')\
.filter(df_crashes['DUI Involved']=='Y')\
.groupBy("CSEF Severity")\
.count()\
.withColumn('Percentage', (F.col('count') / total) * 100)\
.show()

+-------------+-----+------------------+
|CSEF Severity|count|        Percentage|
+-------------+-----+------------------+
|     4: Fatal|   27|15.428571428571427|
|        2: MI|   89|50.857142857142854|
|       1: PDO|   24|13.714285714285715|
|        3: SI|   35|              20.0|
+-------------+-----+------------------+



In [38]:
# 2.d
# Total number of cases not involving DUI/BAC and drugs
total = df_crashes.filter(df_crashes['Drugs Involved'].isNull())\
        .filter((df_crashes['DUI Involved']).isNull()).count()

# Count and percentage for cases not involving DUI/BAC and drugs
df_crashes.filter(df_crashes['Drugs Involved'].isNull())\
.filter((df_crashes['DUI Involved']).isNull())\
.groupBy("CSEF Severity")\
.count()\
.withColumn('Percentage', (F.col('count') / total) * 100)\
.show()

+-------------+-----+------------------+
|CSEF Severity|count|        Percentage|
+-------------+-----+------------------+
|     4: Fatal|  317| 0.461567582521586|
|        2: MI|20484|29.825710916000524|
|       1: PDO|45371| 66.06240626683557|
|        3: SI| 2507|3.6503152346423215|
+-------------+-----+------------------+



When no drugs are involved and the driver tested negative for an illegal level of BAC, the crashses that took place mostly resulted in a property damage only and very few of them were fatal. While, when both drugs were involved and driver tested postive for a high BAC level 50% of the cases resulted in a minor injury and 15% crashes were fatal, which is huge. When only drugs were involved 59% crashes resulted in a minor injury and when the driver tested positive for a high BAC level 52% cases resulted in only property damage and 32% cases resulted in a minor injury. 
We can also observer that highest number of cases were recorded when no drugs were involved and there was no detection of a high BAC level while there were only a few cases when both drugs and alcohol was involved. 

### 2.4 RDDs vs DataFrame vs Spark SQL (20%)

In [39]:
# register the original DataFrame as a temp view so that we can query it using SQL
df_units.createOrReplaceTempView("sql_units")
df_crashes.createOrReplaceTempView("sql_crashes")

In [40]:
# 1.
# Joining crash and units df on REPORT_ID
# Extracting columns date, time of crash and other required columns for suburb Adelaide

In [41]:
%%time
df_joint = df_units.join(df_crashes, df_crashes.REPORT_ID==df_units.REPORT_ID, how='outer')
df_joint.filter(df_joint['Suburb']=='ADELAIDE')\
.select(F.concat('Year',F.lit("-"), 'Month' ,F.lit("-"), 'Day').alias("Date"), 'Time', 'Unit No', 'No Of Cas', 'Sex', 'Age', 'Licence Type')\
.show()

+--------------------+--------+-------+---------+-------+----+------------+
|                Date|    Time|Unit No|No Of Cas|    Sex| Age|Licence Type|
+--------------------+--------+-------+---------+-------+----+------------+
|2016-November-Wed...|01:45 pm|     02|        1|   Male| 072|        null|
|2016-November-Wed...|01:45 pm|     01|        0|   Male| 056|        Full|
|2016-November-Tue...|03:40 pm|     02|        1| Female| 027|        null|
|2016-November-Tue...|03:40 pm|     01|        0|   Male| 056|        null|
|2016-November-Tue...|05:00 pm|     02|        0|Unknown| XXX|     Unknown|
|2016-November-Tue...|05:00 pm|     01|        0| Female| 032|        Full|
|2016-November-Tue...|05:40 pm|     02|        0|   Male| 020|     Unknown|
|2016-November-Tue...|05:40 pm|     01|        0|   Male| 022|     Unknown|
|2016-November-Monday|11:26 pm|     03|        0|   null|null|        null|
|2016-November-Monday|11:26 pm|     02|        0|   Male| 042|        Full|
|2016-Novemb

In [42]:
# 1.
# sql statements for the same tasks

In [43]:
%%time
sql_adelaide = spark.sql('''
    SELECT CONCAT(Year, '-', Month, '-', Day) as Date, Time, `Unit No`, `No Of Cas`, Sex, Age, `Licence Type`
    FROM sql_crashes c join sql_units u 
    ON c.REPORT_ID = u.REPORT_ID
    WHERE Suburb == 'ADELAIDE' 
''')
sql_adelaide.show()

+--------------------+--------+-------+---------+-------+----+------------+
|                Date|    Time|Unit No|No Of Cas|    Sex| Age|Licence Type|
+--------------------+--------+-------+---------+-------+----+------------+
|2016-November-Wed...|01:45 pm|     01|        0|   Male| 056|        Full|
|2016-November-Wed...|01:45 pm|     02|        1|   Male| 072|        null|
|2016-November-Tue...|03:40 pm|     01|        0|   Male| 056|        null|
|2016-November-Tue...|03:40 pm|     02|        1| Female| 027|        null|
|2016-November-Tue...|05:00 pm|     01|        0| Female| 032|        Full|
|2016-November-Tue...|05:00 pm|     02|        0|Unknown| XXX|     Unknown|
|2016-November-Tue...|05:40 pm|     01|        0|   Male| 022|     Unknown|
|2016-November-Tue...|05:40 pm|     02|        0|   Male| 020|     Unknown|
|2016-November-Monday|11:26 pm|     01|        0|Unknown| XXX|     Unknown|
|2016-November-Monday|11:26 pm|     02|        0|   Male| 042|        Full|
|2016-Novemb

In [44]:
# 1. 
# rdd statements for the same task
rdd_units_3 = rdd_units_1.map(lambda x: (x.pop(0), x[:]))
rdd_crashes_1 = rdd_crashes.map(lambda x: x.split(","))
rdd_crashes_3 = rdd_crashes_1.map(lambda x: (x.pop(0), x[:]))

In [45]:
%%time
rdd_joint = rdd_crashes_3.join(rdd_units_3)
rdd_joint_1 = rdd_joint.map(lambda x: (x[1][0][1], x[1][0][9]+'-'+x[1][0][10]+'-'+x[1][0][11], x[1][0][12], x[1][1][1], x[1][1][6], x[1][1][7], x[1][1][10]))
rdd_filter = rdd_joint_1.filter(lambda x: x[0]=='"ADELAIDE"')
rdd_filter.collect()

CPU times: user 47.4 ms, sys: 1.13 ms, total: 48.6 ms
Wall time: 6.33 s


[('"ADELAIDE"',
  '2019-"January"-"Friday"',
  '"12:10 am"',
  '0',
  '"Male"',
  '"036"',
  '"Full"'),
 ('"ADELAIDE"',
  '2019-"January"-"Friday"',
  '"12:10 am"',
  '0',
  '"Male"',
  '"033"',
  '"Full"'),
 ('"ADELAIDE"',
  '2019-"January"-"Thursday"',
  '"12:15 pm"',
  '0',
  '"Male"',
  '"022"',
  '"Full"'),
 ('"ADELAIDE"',
  '2019-"January"-"Thursday"',
  '"12:15 pm"',
  '0',
  '"Female"',
  '"044"',
  '"Full"'),
 ('"ADELAIDE"',
  '2019-"February"-"Tuesday"',
  '"11:05 am"',
  '0',
  '"Male"',
  '"050"',
  ''),
 ('"ADELAIDE"',
  '2019-"February"-"Tuesday"',
  '"11:05 am"',
  '0',
  '"Male"',
  '"041"',
  '"Full"'),
 ('"ADELAIDE"',
  '2019-"February"-"Tuesday"',
  '"07:35 pm"',
  '1',
  '"Male"',
  '"018"',
  '"Learners"'),
 ('"ADELAIDE"', '2019-"February"-"Tuesday"', '"07:35 pm"', '0', '', '', ''),
 ('"ADELAIDE"',
  '2019-"March"-"Saturday"',
  '"03:41 pm"',
  '0',
  '"Male"',
  '"018"',
  '"Provisional 2"'),
 ('"ADELAIDE"',
  '2019-"March"-"Saturday"',
  '"03:41 pm"',
  '0',
  '"

In [46]:
# 2.
# Joining crash and units df on REPORT_ID
# Extracting Suburb and Total Cas for unlicenced drivers and grouping by suburbs

In [47]:
%%time
df_joint = df_units.join(df_crashes, df_crashes.REPORT_ID==df_units.REPORT_ID, how='outer')
df_joint.filter(df_joint['Licence Type']=='Unlicenced')\
.groupBy("Suburb")\
.agg(F.sum("No Of Cas").alias("Total Casualities"))\
.show()

+-------------------+-----------------+
|             Suburb|Total Casualities|
+-------------------+-----------------+
|      FLINDERS PARK|              4.0|
|       POOGINAGORIC|              1.0|
|     TEA TREE GULLY|              1.0|
|            HACKHAM|              3.0|
|   MEDINDIE GARDENS|              0.0|
|           WISANGER|              1.0|
|            CUMMINS|              0.0|
|       BASKET RANGE|              0.0|
|MURRAY BRIDGE SOUTH|              0.0|
|      GILLES PLAINS|              6.0|
|             HAWKER|              0.0|
|           BEAUFORT|              1.0|
|             MAGILL|              4.0|
|            ECHUNGA|              1.0|
|            CULTANA|              1.0|
|        EDWARDSTOWN|              3.0|
|        RISDON PARK|              1.0|
|          THORNGATE|              0.0|
|       ANDREWS FARM|             10.0|
|       TORRENSVILLE|              3.0|
+-------------------+-----------------+
only showing top 20 rows

CPU times: use

In [48]:
# 2.
# sql statements for the same task

In [49]:
%%time
a = spark.sql('''
    SELECT Suburb, sum(`No Of Cas`) as `Total Casualities`
    FROM sql_crashes c join sql_units u 
    ON c.REPORT_ID = u.REPORT_ID
    where `Licence Type` == "Unlicenced"
    Group By Suburb
''')
a.show()

+-------------------+-----------------+
|             Suburb|Total Casualities|
+-------------------+-----------------+
|      FLINDERS PARK|              4.0|
|       POOGINAGORIC|              1.0|
|     TEA TREE GULLY|              1.0|
|            HACKHAM|              3.0|
|   MEDINDIE GARDENS|              0.0|
|           WISANGER|              1.0|
|            CUMMINS|              0.0|
|       BASKET RANGE|              0.0|
|MURRAY BRIDGE SOUTH|              0.0|
|      GILLES PLAINS|              6.0|
|             HAWKER|              0.0|
|           BEAUFORT|              1.0|
|             MAGILL|              4.0|
|            ECHUNGA|              1.0|
|            CULTANA|              1.0|
|        EDWARDSTOWN|              3.0|
|        RISDON PARK|              1.0|
|          THORNGATE|              0.0|
|       ANDREWS FARM|             10.0|
|       TORRENSVILLE|              3.0|
+-------------------+-----------------+
only showing top 20 rows

CPU times: use

In [50]:
# 2.
# rdd statements for the same task

In [51]:
%%time
rdd_joint = rdd_crashes_3.join(rdd_units_3)
rdd_join_1 = rdd_joint.map(lambda x: (x[1][0][1], x[1][1][1], x[1][1][10]))
rdd_join_2 = rdd_join_1.filter(lambda x: x[2] == '"Unlicenced"').map(lambda x: (x[0],int(x[1])))
rdd_join_3 = rdd_join_2.reduceByKey(lambda a,b : a+b) 
rdd_join_3.collect()

CPU times: user 44.7 ms, sys: 0 ns, total: 44.7 ms
Wall time: 6.47 s


[('"SOUTH PLYMPTON"', 4),
 ('"ELIZABETH DOWNS"', 6),
 ('"MILE END"', 4),
 ('"PORT LINCOLN"', 2),
 ('"BOWDEN"', 0),
 ('"MODBURY"', 2),
 ('"WINGFIELD"', 3),
 ('"TORRENSVILLE"', 3),
 ('"SELLICKS HILL"', 0),
 ('"MENINGIE"', 2),
 ('"NORTHFIELD"', 1),
 ('"BIRKENHEAD"', 2),
 ('"ROSE PARK"', 0),
 ('"PORT AUGUSTA WEST"', 2),
 ('"WHYALLA NORRIE"', 5),
 ('"GOOLWA BEACH"', 1),
 ('"GAWLER BELT"', 0),
 ('"TATACHILLA"', 1),
 ('"YAMBA"', 1),
 ('"ELIZABETH GROVE"', 5),
 ('"SMITHFIELD"', 3),
 ('"GLENGOWRIE"', 1),
 ('"HILLIER"', 1),
 ('"BINNUM"', 1),
 ('"MARION"', 3),
 ('"HAWKER"', 0),
 ('"PARALOWIE"', 2),
 ('"BRIGHTON"', 2),
 ('"YALATA"', 1),
 ('"OAKLANDS PARK"', 1),
 ('"CURRENCY CREEK"', 5),
 ('"MEDINDIE GARDENS"', 0),
 ('"CRYSTAL BROOK"', 0),
 ('"OLARY"', 0),
 ('"CUNLIFFE"', 1),
 ('"BLAKEVIEW"', 3),
 ('"CHANDLERS HILL"', 1),
 ('"SEMAPHORE SOUTH"', 3),
 ('"BOLTO"', 0),
 ('"SEAFORD RISE"', 0),
 ('"RENMARK SOUTH"', 1),
 ('"ANANGU PITJANTJATJARA YANKUNYTJATJARA"', 0),
 ('"HACKHAM"', 3),
 ('"ENCOUNTER BAY"

After looking at the time statistics we can conclude that the sql query performs the best while the rdd commands work the slowest for the given tasks/\. 